In [61]:
import os
import sys
import os.path as osp
import torch
from torchvision.transforms import Normalize
import numpy as np
import cv2
import argparse
import json
import _pickle as pkl
from datetime import datetime

from demo.demo_options import DemoOptions
from bodymocap.body_mocap_api import BodyMocap
from bodymocap.body_bbox_detector import BodyPoseEstimator
import mocap_utils.demo_utils as demo_utils
import mocap_utils.general_utils as gnu
from mocap_utils.timer import Timer

import renderer.image_utils as imu
from renderer.screen_free_visualizer import Visualizer
from renderer import p3d_renderer

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [76]:
from os.path import exists, join, split
from glob import glob
from psbody.mesh import Mesh, MeshViewer, MeshViewers

from MultiGarmentNetwork.utils.smpl_paths import SmplPaths
from MultiGarmentNetwork.lib.ch_smpl import Smpl
from MultiGarmentNetwork.dress_SMPL import load_smpl_from_file, pose_garment, dress
from MultiGarmentNetwork.utils.interpenetration_ind import remove_interpenetration_fast

from IPython.display import Image

In [3]:
# Set bbox detector
body_bbox_detector = BodyPoseEstimator()

use_smplx = False

# Set mocap regressor
checkpoint_path = './extra_data/body_module/pretrained_weights/2020_05_31-00_50_43-best-51.749683916568756.pt'
# checkpoint_path = './extra_data/body_module/pretrained_weights/smplx-03-28-46060-w_spin_mlc3d_46582-2089_2020_03_28-21_56_16.pt'
smpl_dir = './extra_data/smpl/'
body_mocap = BodyMocap(checkpoint_path, smpl_dir, device, use_smplx=use_smplx)

visualizer = Visualizer('pytorch3d')

Loading Body Pose Estimator


In [50]:
# MGN
path = '../Multi-Garment_dataset/'
garment_classes = ['Pants', 'ShortPants', 'ShirtNoCoat', 'TShirtNoCoat', 'LongCoat']
gar_dict = {}
for gar in garment_classes:
    gar_dict[gar] = glob(join(path, '*', gar + '.obj'))

dp = SmplPaths()
# vt, ft = dp.get_vt_ft_hres()

## This file contains correspondances between garment vertices and smpl body
fts_file = 'MultiGarmentNetwork/assets/garment_fts.pkl'
vert_indices, fts = pkl.load(open(fts_file, 'rb') , encoding='latin1')
# fts['naked'] = ft

## Choose any garmet type as source
garment_type = 'Pants'
index = np.random.randint(0, len(gar_dict[garment_type]))   ## Randomly pick from the digital wardrobe
path = split(gar_dict[garment_type][index])[0]
    
garment_org_body_unposed = load_smpl_from_file(join(path, 'registration.pkl'))
garment_org_body_unposed.pose[:] = 0
garment_org_body_unposed.trans[:] = 0
garment_org_body_unposed = Mesh(garment_org_body_unposed.v, garment_org_body_unposed.f)

garment_unposed = Mesh(filename=join(path, garment_type + '.obj'))
garment_tex = join(path, 'multi_tex.jpg')
tgt_body = Mesh(smpl.r, smpl.f)

vert_inds = vert_indices[garment_type]
garment_unposed.set_texture_image(garment_tex)

In [55]:
start_frame = 0
cur_frame = start_frame
video_frame = 0
timer = Timer()

class Args():
    def __init__(self):
        self.input_path = './sample_data/han_short.mp4'
        self.input_type = 'video'
        self.save_frame = False
        self.out_dir = './mocap_output'
        self.end_frame = float('inf')
        self.save_bbox_output = False
        self.single_person = False
        self.no_display = True
        self.save_pred_pkl = True
        self.use_smplx = True
        self.save_mesh = True
        self.no_video_out = False

args = Args()

use_smplx = args.use_smplx
        
input_type, input_data = demo_utils.setup_input(args)

while True:
    timer.tic()
    # load data
    if input_type == 'video':      
        _, img_original_bgr = input_data.read()
        if video_frame < cur_frame:
            video_frame += 1
            continue
        # save the obtained video frames
        image_path = osp.join(args.out_dir, "frames", f"{cur_frame:05d}.jpg")
        if img_original_bgr is not None:
            video_frame += 1
            if args.save_frame:
                gnu.make_subdir(image_path)
                cv2.imwrite(image_path, img_original_bgr)
    else:
        assert False, "Unknown input_type"

    cur_frame +=1
    if img_original_bgr is None or cur_frame > args.end_frame:
        break   
    print("--------------------------------------")

    body_pose_list, body_bbox_list = body_bbox_detector.detect_body_pose(img_original_bgr)

    if len(body_bbox_list) < 1: 
        print(f"No body detected: {image_path}")
        continue

    #Sort the bbox using bbox size 
    # (to make the order as consistent as possible without tracking)
    bbox_size =  [(x[2] * x[3]) for x in body_bbox_list]
    idx_big2small = np.argsort(bbox_size)[::-1]
    body_bbox_list = [ body_bbox_list[i] for i in idx_big2small ]
    if args.single_person and len(body_bbox_list) > 0:
        body_bbox_list = [body_bbox_list[0], ]       

    # Body Pose Regression
    pred_output_list = body_mocap.regress(img_original_bgr, body_bbox_list)
    assert len(body_bbox_list) == len(pred_output_list)
    
    # extract mesh for rendering (vertices in image space and faces) from pred_output_list
#     pred_mesh_list = demo_utils.extract_mesh_from_output(pred_output_list)
    
#     smpl = Smpl(dp.get_hres_smpl_model_data())
#     smpl.pose[:] = pred_output_list[0]['pred_body_pose'][0] * 0.05
#     smpl.betas[:] = pred_output_list[0]['pred_betas'][0] * 0.01
#     smpl.trans[:] = 0
    
#     new_garment = dress(smpl, garment_org_body_unposed, garment_unposed, vert_inds, garment_tex)
    
#     v_f = [{'vertices': new_garment.v, 'faces': new_garment.f.astype('int32')}]
    # visualization
    
#     renderer = Pytorch3dRenderer(480, )
#     renderer.render(verts=new_garment.v, faces=new_garment.f, bg_img=img_original_bgr)
    
    res_img = visualizer.visualize(
        img_original_bgr,
        pred_mesh_list = pred_mesh_list, 
        body_bbox_list = body_bbox_list)

    # save result image
    if args.out_dir is not None:
        demo_utils.save_res_img(args.out_dir, image_path, res_img)

    timer.toc(bPrint=True,title="Time")
    print(f"Processed : {image_path}")
    break

#save images as a video
# if not args.no_video_out and input_type in ['video', 'webcam']:
#     demo_utils.gen_video_out(args.out_dir, args.seq_name)

# cv2.destroyAllWindows()

--------------------------------------
310 453 115 473
Using medium size renderer
428
Visualization saved: ./mocap_output/rendered/00000.jpg
Time: 0.53 sec/frame, FPS 1.90
Processed : ./mocap_output/frames/00000.jpg


In [91]:
pred_output_list[0]['pred_body_pose']

array([[ 2.91917896e+00,  4.58017327e-02,  1.53551832e-01,
        -9.22530651e-01,  8.30429792e-02,  3.82127285e-01,
        -8.67569566e-01, -9.41842701e-03, -3.08711380e-01,
         5.51593065e-01, -2.62972210e-02,  1.17724165e-02,
         1.55291820e+00,  9.38382372e-02, -2.03269675e-01,
         1.60624826e+00, -1.72855295e-02,  1.70969218e-01,
        -5.60724996e-02, -2.51412056e-02, -6.79262029e-03,
        -1.70694739e-01,  7.73771927e-02, -6.54179528e-02,
        -2.13481039e-01, -2.78314129e-02, -1.45036180e-03,
        -3.69013101e-02, -1.13427769e-02, -8.66152346e-03,
        -2.59527802e-01,  6.41327873e-02,  1.68739066e-01,
        -2.42282629e-01, -1.39641911e-01, -1.29240543e-01,
        -3.32024634e-01, -9.92928073e-02, -7.45689273e-02,
         5.03234453e-02, -2.15491727e-01, -3.47436190e-01,
         3.79543081e-02,  1.83946118e-01,  3.41378152e-01,
        -4.32795621e-02, -8.62272084e-02,  1.32593354e-02,
         9.13528800e-02, -3.66373211e-01, -8.13994765e-0

In [79]:
smpl.pose[:] = np.random.randn(72) *0.05
smpl.betas[:] = np.random.randn(10) *0.01
smpl.trans[:] = 0
tgt_body = Mesh(smpl.r, smpl.f)

In [80]:
vert_inds = vert_indices[garment_type]
garment_unposed.set_texture_image(garment_tex)

In [87]:
garment_unposed.v

array([[ 0.073349,  0.261848,  0.031269],
       [ 0.08271 ,  0.270592,  0.012117],
       [ 0.078152,  0.277105,  0.0038  ],
       ...,
       [-0.102649,  0.121517,  0.09446 ],
       [-0.094741,  0.115864,  0.102627],
       [-0.099849,  0.114106,  0.098278]])

In [88]:
new_garment.v

array([[ 0.06929031,  0.26804627,  0.01835646],
       [ 0.07501148,  0.27842027, -0.00282235],
       [ 0.06833248,  0.28578541, -0.01007526],
       ...,
       [-0.09944983,  0.11889105,  0.13553003],
       [-0.0898118 ,  0.11403581,  0.14393182],
       [-0.09602736,  0.11176193,  0.1400399 ]])

In [81]:
new_garment = dress(smpl, garment_org_body_unposed, garment_unposed, vert_inds, garment_tex)

In [82]:
input_size = 1920
renderer = Pytorch3dRenderer(img_size=input_size, mesh_color=[0.8, 0.53, 0.53])

In [83]:
rend_img = np.ones((input_size, input_size, 3))

In [84]:
res_img = renderer.render(verts=new_garment.v * 1920, faces=new_garment.f.astype('int32'), bg_img=img_original_bgr)

-1026 905 -588 580
Using large size renderer
1098


In [85]:
demo_utils.save_res_img(args.out_dir, image_path, res_img)

Visualization saved: ./mocap_output/rendered/00000.jpg
